In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:

import json
import os
from copy import deepcopy
from time import time

import numpy as np
import pandas as pd

from backtest import backtest
from downloader import Downloader, load_hlc_cache
from plotting import make_table
from procedures import (
    prepare_backtest_config,
)
from pure_funcs import (
    analyze_fills,
)

In [7]:
class Args:
    def __init__(self,
                 backtest_config_path="configs/backtest/myconfig_v03.hjson",
                 exchange="bitget",
                 symbols="SEIUSDT",
                 market_type="futures",
                 user="bitget_01",
                 start_date="2021-01-13",
                 end_date="2024-01-29",
                 tarting_balance=1000.0,
                 starting_configs="",
                 base_dir="backtests",
                 ohlcv="y"  # set to "n" to use 1s candles
                 ):
        self.backtest_config_path = backtest_config_path
        self.exchange = exchange
        self.symbols = symbols
        self.market_type = market_type
        self.user = user
        self.start_date = start_date
        self.end_date = end_date
        self.starting_balance = tarting_balance
        self.starting_configs = starting_configs
        self.base_dir = base_dir
        self.ohlcv = ohlcv

In [8]:
configs_path = "configs/live/au_20240213_multi_rec_grid"

pairs = os.listdir(configs_path)
start_date = "2021-01-13"
end_date = " 2024-01-29"

counter_i = 0
results = []
for pair in pairs:
    pair_name = pair.split(".")[0]
    config = prepare_backtest_config(
        Args(
            symbols=pair_name,
            start_date=start_date,
            end_date=end_date
        )
    )

    config["adg_n_subdivisions"] = 20  # see configs/backtest/default.hjson for details
    config["inverse"] = False
    sts = time()
    if config["ohlcv"]:
        data = await load_hlc_cache(
            config["symbol"],
            config["inverse"],
            config["start_date"],
            config["end_date"],
            base_dir=config["base_dir"],
            spot=config["spot"],
            exchange=config["exchange"],
        )
    else:
        downloader = Downloader(config)
        data = await downloader.get_sampled_ticks()
    timestamps = data[:, 0]
    config["n_days"] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)
    print(f"millis to load {len(data)} ticks {(time() - sts) * 1000:.0f}ms of {pair_name}")

    # choose a slice on which to test
    wsize_days = 3650
    ts = int(data[-1][0] - 60 * 60 * 24 * 1000 * wsize_days)
    idx = np.argmax(data[:, 0] >= ts)
    dataslice = data[idx:]
    print(f"len(dataslice)={len(dataslice)}")

    with open(os.path.join(configs_path, pair), "r") as f:
        tune_pairs = json.load(f)

    for w_exp in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0, 6.0, 8.0, 10.0, 15.0, 20.0, 30.0]:
        
        tune_dict = {}
        for op_type in ["long", "short"]:
            tune_pairs[op_type]["wallet_exposure_limit"] = w_exp
            tune_dict.update({(op_type, k): v for k, v in tune_pairs[op_type].items()})
            

        config_to_test = {**config, **tune_pairs}
    
        sts = time()
        fills_long, fills_short, stats = backtest(config_to_test, dataslice)
        elapsed = time() - sts
    
        longs, shorts, sdf, analysis = analyze_fills(fills_long, fills_short, stats, config_to_test)
        table = make_table({**config_to_test, **analysis})
        
        print(counter_i, pair, w_exp)
        print(table)
        print("\n")
        
        dict_temp = deepcopy(
            {
                **{"row": counter_i, "pair": pair_name, "wallet_exposure": w_exp},
                **analysis,
                **tune_dict
            }
        )
        results.append(dict_temp)
        counter_i += 1
        
        if counter_i % 20 == 0:
            results_df = pd.DataFrame(results)
            results_df.to_csv("/home/raul/Escritorio/results_multi_df.csv")
    #     break
    # break
    
results_df = pd.DataFrame(results)
results_df.to_csv("/home/raul/Escritorio/results_multi_df.csv")


fetching market_specific_settings for 1000FLOKIUSDT...


Traceback (most recent call last):
  File "/home/tdb/git/passivbot/procedures.py", line 206, in add_market_specific_settings
    market_specific_settings = fetch_market_specific_settings(config)
  File "/home/tdb/git/passivbot/procedures.py", line 758, in fetch_market_specific_settings
    raise Exception(f"unknown symbol {symbol}")
Exception: unknown symbol 1000FLOKIUSDT



failed to fetch market_specific_settings for symbol 1000FLOKIUSDT unknown symbol 1000FLOKIUSDT 


Exception: failed to load cached market_specific_settings for symbol 1000FLOKIUSDT